In [13]:
import importlib
import analisis_pob_mon_lab
importlib.reload((analisis_pob_mon_lab))

<module 'analisis_pob_mon_lab' from 'E:\\papx\\end_to_end_ml\\nb_pr\\enahopy\\examples\\investigacion\\analisis_pob_mon_lab.py'>

In [14]:
# Instalación (si es necesario)
# !pip install enahopy


import enahopy


# ========== USANDO ENAHOPY ENAHO LOADER ==========
from enahopy.loader import ENAHODataDownloader
from enahopy.loader.io import ENAHOLocalReader


# ========== USANDO ENAHOPY's ENAHOModuleMerger ==========
from enahopy.merger import ENAHOModuleMerger
from enahopy.merger.config import ModuleMergeConfig, ModuleMergeLevel
import logging


# ========== USANDO ENAHOPY ENAHO NULL_ANALYSIS ==========
from enahopy.null_analysis import ENAHONullAnalyzer

import pandas as pd
import numpy as np
from datetime import datetime
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List, Tuple, Optional


warnings.filterwarnings('ignore')

print(f"enahopy versión: {enahopy.__version__}")
print(f"Inicio del análisis: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

enahopy versión: 0.6.0
Inicio del análisis: 2025-10-29 19:30:19


In [15]:
# ========== CONFIGURACIÓN ENAHOModuleMerger ==========

# Configurar el merger para nivel individual (persona)
config = ModuleMergeConfig(merge_level=ModuleMergeLevel.PERSONA)
logger = logging.getLogger('enaho_merger')
merger = ENAHOModuleMerger(config, logger)


# Configurar el merger para nivel hogar
config_hogar = ModuleMergeConfig(merge_level=ModuleMergeLevel.HOGAR)
merger_hogar = ENAHOModuleMerger(config_hogar, logger)




In [17]:
def descargar_modulos_enaho(año: str = '2024') -> Dict:
    """
    Descarga múltiples módulos ENAHO usando enahopy.

    Args:
        año: Año de los datos ENAHO

4

        Dict con datasets por módulo
    """
    from enahopy.loader import ENAHODataDownloader

    print(f"\n{'=' * 70}")
    print(f"DESCARGANDO MÓDULOS ENAHO {año}".center(70))
    print(f"{'=' * 70}\n")

    modulos_interes = {
        "01": "Características de vivienda y hogar",
        "02": "Características de los Miembros del Hogar",
        "03": "Educación",
        "04": "Salud",
        "05": "Empleo e Ingresos",
        "34": "Sumarias (Variables Calculadas)"
    }

    downloader = ENAHODataDownloader(verbose=True)

    print("Módulos a descargar:")
    for code, desc in modulos_interes.items():
        print(f"  {code}: {desc}")

    data_multi = downloader.download(
        modules=list(modulos_interes.keys()),
        years=[año],
        output_dir=r"E:\papx\end_to_end_ml\nb_pr\enahopy\examples\investigacion\data",
        decompress=True,
        only_dta=True,
        load_dta=True,
        parallel=True,  # ¡Descarga paralela!
        max_workers=3,  # 3 módulos en paralelo
        verbose=True
    )
    print(f"\n✓ Descarga completada")
    return data_multi, modulos_interes


def extraer_datasets(data_multi: Dict, año: str = '2024') -> Dict:
    """
    Extrae datasets individuales del diccionario descargado.
    """
    print("Extrayendo datasets...")

    # Variables básicas para cada dataset
    vars_hogar = ['conglome', 'vivienda', 'hogar', 'p102', 'p103', 'p104', 'p105a', 'p110', 'p111a', 'p113a', 'p1121',
                  'p1144']
    vars_persona = ['conglome', 'vivienda', 'hogar', 'codperso', 'p203', 'p203b', 'p207', 'p208a', 'p209']
    vars_educa = ['conglome', 'vivienda', 'hogar', 'codperso', 'p301a', 'p301b', 'p301c', 'p307']
    vars_salud = ['conglome', 'vivienda', 'hogar', 'codperso', 'p401', 'p4191', 'p4192', 'p4193', 'p4194',
                  'p4195','p4196', 'p4197', 'p4198', 'p401h1', 'p401h2', 'p401h3', 'p401h4', 'p401h5', 'p401h6', 'p401f', 'p401g']
    vars_empleo = ['conglome', 'vivienda', 'hogar', 'codperso', 'ocu500', 'p507', 'i524e1', 'i524a1', 'p513t', 'p510a1',
                   'p512a', 'p558a5', 'i530a', 'p530a', 'i538e1', 'p538e1', 'i541a', 'p541a', 'p505r4', 'p506r4',
                   'p511a']
    vars_sumaria = ['conglome', 'vivienda', 'hogar', 'ubigeo', 'dominio', 'estrato', 'pobreza', 'inghog2d', 'gashog2d',
                    'factor07', 'mieperho', 'linea', 'linpe']

    # Cargar datasets
    data_caracteristica_hogar = data_multi[(año, '01')]['enaho01-' + año + '-100'][vars_hogar]
    data_caracteristica_persona = data_multi[(año, '02')]['enaho01-' + año + '-200'][vars_persona]
    data_educacion = data_multi[(año, '03')]['enaho01a-' + año + '-300'][vars_educa]
    data_salud = data_multi[(año, '04')]['enaho01a-' + año + '-400'][vars_salud]
    data_empleo = data_multi[(año, '05')]['enaho01a-' + año + '-500'][vars_empleo]
    data_sumaria = data_multi[(año, '34')]['sumaria-' + año][vars_sumaria]

    datasets = {
        'hogar': data_caracteristica_hogar,
        'persona': data_caracteristica_persona,
        'educacion': data_educacion,
        'salud': data_salud,
        'empleo': data_empleo,
        'sumaria': data_sumaria
    }

    # Mostrar resumen simple
    for nombre, df in datasets.items():
        print(f"  {nombre}: {df.shape[0]:,} filas × {df.shape[1]} columnas")

    return datasets


In [18]:

def fusionar_modulos_individuales(data_persona: pd.DataFrame,
                                  data_educacion: pd.DataFrame,
                                  data_salud: pd.DataFrame,
                                  data_empleo: pd.DataFrame) -> pd.DataFrame:
    """
    Fusiona todos los módulos individuales en un único DataFrame.
    Usa enahopy ENAHOModuleMerger para hacer merge correcto.
    """
    from enahopy.merger import ENAHOModuleMerger
    from enahopy.merger.config import ModuleMergeConfig, ModuleMergeLevel

    print(f"\n{'=' * 70}")
    print("PASO 1: FUSIÓN DE MÓDULOS INDIVIDUALES".center(70))
    print(f"{'=' * 70}\n")

    print("Fusionando datos individuales (persona + educación + salud + empleo)...\n")

    # Configurar merger para nivel PERSONA
    config = ModuleMergeConfig(merge_level=ModuleMergeLevel.PERSONA)
    logger_merger = logging.getLogger('enaho_merger')
    merger = ENAHOModuleMerger(config, logger_merger)

    modules_dict = {
        '02': data_persona,  # Características de miembros del hogar
        '04': data_salud,  # Salud
        '03': data_educacion,  # Educación
        '05': data_empleo  # Empleo
    }

    print("📦 Usando ENAHOModuleMerger de enahopy para fusionar módulos 02, 03, 04, 05...")
    merge_result = merger.merge_multiple_modules(
        modules_dict=modules_dict,
        base_module='02',  # Características como base
        merge_config=config
    )
    # Realizar merge usando enahopy
    data_individuos = merge_result.merged_df

    print(f"\n✓ Módulos individuales fusionados: {data_individuos.shape}")
    return data_individuos


In [19]:
def merge_con_hogar(data_caracteristica_hogar: pd.DataFrame,
                      data_sumaria: pd.DataFrame) -> pd.DataFrame:
    """
    Merge de datos agregados de nivel individual con sumaria (módulo 01).
    Usa enahopy para hacer merge correcto.
    """
    from enahopy.merger import ENAHOModuleMerger
    from enahopy.merger.config import ModuleMergeConfig, ModuleMergeLevel


    # Configurar merger para nivel HOGAR
    config_hogar = ModuleMergeConfig(merge_level=ModuleMergeLevel.HOGAR)
    logger_merger = logging.getLogger('enaho_merger')
    merger_hogar = ENAHOModuleMerger(config_hogar, logger_merger)



    print("\n📦 Usando ENAHOModuleMerger de enahopy para fusionar sumaria con caracteristicas del hogar...")
    merge_con_hogar = merger_hogar.merge_modules(
        left_df=data_sumaria,           # Sumaria como base (left)
        right_df=data_caracteristica_hogar,         # Datos agregados (right)
        left_module='34',           # Módulo sumaria
        right_module='01',    # Identificador para datos agregados
        merge_config=config_hogar
    )

    # Extraer el DataFrame resultante
    data_con_hogar = merge_con_hogar.merged_df

    print(f"\n✓ Módulos a nivel Hogares fusionados: {data_con_hogar.shape}")
    return data_con_hogar

In [20]:
# PASO 1: Descargar y extraer la información de INEI con ENAHOPY

data_multi, modulos = descargar_modulos_enaho('2024')
datasets = extraer_datasets(data_multi, '2024')

# PASO 2: Fusionar módulos individuales

data_individuos = fusionar_modulos_individuales(
    datasets['persona'],
    datasets['educacion'],
    datasets['salud'],
    datasets['empleo']
)

# PASO 3: Fusionar módulos de hogares 1 y sumaria

data_hogares = merge_con_hogar(
    datasets['hogar'],
    datasets['sumaria']
)


                    DESCARGANDO MÓDULOS ENAHO 2024                    

2025-10-29 19:30:37 | INFO     | enahopy | ENAHOPY logging initialized
2025-10-29 19:30:37 | DEBUG    | enahopy.loader.core.cache | No expired cache entries to clean
Módulos a descargar:
  01: Características de vivienda y hogar
  02: Características de los Miembros del Hogar
  03: Educación
  04: Salud
  05: Empleo e Ingresos
  34: Sumarias (Variables Calculadas)
2025-10-29 19:30:37 | INFO     | enaho_downloader | === Iniciando descarga ENAHO corte transversal ===
2025-10-29 19:30:37 | INFO     | enaho_downloader | Módulos: ['01', '02', '03', '04', '05', '34']
2025-10-29 19:30:37 | INFO     | enaho_downloader | Años: ['2024']
2025-10-29 19:30:37 | INFO     | enaho_downloader | Directorio: E:\papx\end_to_end_ml\nb_pr\enahopy\examples\investigacion\data
2025-10-29 19:30:37 | INFO     | enaho_downloader | Total de descargas programadas: 6
2025-10-29 19:30:37 | INFO     | enaho_downloader | Descarga paralela con 3 wo

Descargando modulo_01_2024.zip:  30%|██▉       | 2.49M/8.38M [00:01<00:03, 1.77MiB/s]
Descargando modulo_03_2024.zip:   0%|          | 0.00/11.9M [00:00<?, ?iB/s]
Descargando modulo_01_2024.zip:  32%|███▏      | 2.69M/8.38M [00:01<00:03, 1.55MiB/s][A
Descargando modulo_01_2024.zip:  34%|███▍      | 2.86M/8.38M [00:01<00:03, 1.42MiB/s][A
Descargando modulo_01_2024.zip:  36%|███▋      | 3.05M/8.38M [00:02<00:03, 1.35MiB/s][A
Descargando modulo_01_2024.zip:  38%|███▊      | 3.19M/8.38M [00:02<00:04, 1.22MiB/s]A
Descargando modulo_01_2024.zip:  43%|████▎     | 3.59M/8.38M [00:02<00:04, 1.20MiB/s]A
Descargando modulo_01_2024.zip:  45%|████▌     | 3.80M/8.38M [00:02<00:03, 1.39MiB/s]A
Descargando modulo_01_2024.zip:  47%|████▋     | 3.96M/8.38M [00:02<00:03, 1.42MiB/s]A
Descargando modulo_01_2024.zip:  50%|████▉     | 4.18M/8.38M [00:02<00:02, 1.61MiB/s]A
Descargando modulo_01_2024.zip:  53%|█████▎    | 4.41M/8.38M [00:02<00:02, 1.74MiB/s]
Descargando modulo_01_2024.zip:  58%|█████▊    | 4.8

2025-10-29 19:31:05 | INFO     | enaho_downloader | Descarga completada: modulo_01_2024.zip (8.0 MB)




Descargando modulo_02_2024.zip:  21%|██        | 1.22M/5.79M [00:00<00:04, 1.12MiB/s]
Descargando modulo_03_2024.zip:  40%|███▉      | 4.70M/11.9M [00:04<00:05, 1.25MiB/s]

2025-10-29 19:31:05 | INFO     | enaho_downloader | Extraídos 1 archivos en: E:\papx\end_to_end_ml\nb_pr\enahopy\examples\investigacion\data\modulo_01_2024
2025-10-29 19:31:05 | INFO     | enaho_downloader | Archivo ZIP eliminado: modulo_01_2024.zip


...

2025-10-29 19:32:13 | INFO     | enaho_downloader | Extraídos 5 archivos en: E:\papx\end_to_end_ml\nb_pr\enahopy\examples\investigacion\data\modulo_05_2024



Descargando modulo_34_2024.zip:  61%|██████    | 9.72M/15.9M [00:32<00:12, 503kiB/s]

2025-10-29 19:32:13 | INFO     | enaho_downloader | Archivo ZIP eliminado: modulo_05_2024.zip



Descargando modulo_34_2024.zip:  61%|██████▏   | 9.77M/15.9M [00:32<00:12, 487kiB/s]
Descargando modulo_34_2024.zip:  62%|██████▏   | 9.83M/15.9M [00:32<00:13, 438kiB/s]
Descargando modulo_34_2024.zip:  62%|██████▏   | 9.88M/15.9M [00:32<00:13, 445kiB/s]
Descargando modulo_34_2024.zip:  62%|██████▏   | 9.96M/15.9M [00:32<00:11, 528kiB/s]
Descargando modulo_34_2024.zip:  63%|██████▎   | 10.1M/15.9M [00:32<00:08, 667kiB/s]
Descargando modulo_34_2024.zip:  64%|██████▎   | 10.2M/15.9M [00:32<00:08, 665kiB/s]
Descargando modulo_34_2024.zip:  64%|██████▍   | 10.2M/15.9M [00:33<00:08, 648kiB/s]
Descargando modulo_34_2024.zip:  65%|██████▍   | 10.3M/15.9M [00:33<00:08, 677kiB/s]
Descargando modulo_34_2024.zip:  65%|██████▌   | 10.4M/15.9M [00:33<00:07, 756kiB/s]
Descargando modulo_34_2024.zip:  66%|██████▌   | 10.5M/15.9M [00:33<00:07, 751kiB/s]
Descargando modulo_34_2024.zip:  66%|██████▋   | 10.6M/15.9M [00:33<00:07, 696kiB/s]
Descargando modulo_34_2024.zip:  67%|██████▋   | 10.7M/15.9M [00

2025-10-29 19:32:46 | INFO     | enaho_downloader | Descarga completada: modulo_34_2024.zip (15.2 MB)
2025-10-29 19:32:51 | INFO     | enaho_downloader | Extraídos 2 archivos en: E:\papx\end_to_end_ml\nb_pr\enahopy\examples\investigacion\data\modulo_34_2024
2025-10-29 19:32:51 | INFO     | enaho_downloader | Archivo ZIP eliminado: modulo_34_2024.zip
2025-10-29 19:33:08 | INFO     | enaho_downloader | Archivo cargado: sumaria-2024-12g.dta (33691 filas)
2025-10-29 19:33:11 | INFO     | enaho_downloader | Archivo cargado: enaho01a-2024-400.dta (110451 filas)
2025-10-29 19:33:17 | INFO     | enaho_downloader | Archivo cargado: sumaria-2024.dta (33691 filas)
2025-10-29 19:33:22 | INFO     | enaho_downloader | Archivo cargado: enaho01a-2024-500.dta (85992 filas)
2025-10-29 19:33:22 | INFO     | enaho_downloader | Archivo cargado: enaho_tabla_ciiu_rev3.dta (296 filas)
2025-10-29 19:33:22 | INFO     | enaho_downloader | Archivo cargado: enaho_tabla_ciiu_rev4.dta (419 filas)
2025-10-29 19:33:22

In [21]:
from analisis_pob_mon_lab import pipeline_completo, analisis_descriptivo_ponderado


df_final = pipeline_completo(
      df_individuos=data_individuos,
      df_hogares_enaho=data_hogares,
      ruta_ciiu=r'E:\papx\end_to_end_ml\nb_pr\enahopy\examples\investigacion\data\modulo_05_2024\enaho_tabla_ciiu_rev4.dta',
      ruta_cno=r'E:\papx\end_to_end_ml\nb_pr\enahopy\examples\investigacion\data\modulo_05_2024\enaho_tabla_cno_2015.dta'
  )
#
# resultados = analisis_descriptivo_ponderado(
#     df=df_final,
#     var_pobreza='es_pobre_monetario',
#     peso='factor07'
# )


INICIANDO PIPELINE COMPLETO DE PROCESAMIENTO

→ Cargando tablas CIIU y CNO...
  ✓ CIIU: 52 códigos cargados
  ✓ CNO: 46 códigos cargados

→ Limpiando variables individuales...
  ✓ p208a: 113755 valores válidos
  ✓ p513t: 62874 valores válidos
  ✓ i524e1: 25327 valores válidos
  ✓ p301b: 100072 valores válidos
  ✓ p301c: 23740 valores válidos

→ Creando características individuales...
  • Calculando informalidad laboral...
    ✓ 43078/61878 informales (69.6%)
  • Identificando personas con discapacidad...
    ✓ 6,052 personas con discapacidad (5.1%)
  • Identificando personas con seguro de salud...
    ✓ Encontradas 8 columnas de seguro: p4191, p4192, p4193, p4194, p4195, p4196, p4197, p4198
    DEBUG - Valores únicos en p4191:
                 'no' :   85,929 ( 73.0%)
            'essalud' :   24,422 ( 20.7%)
                  nan :    7,370 (  6.3%)
      • essalud: 24,422 personas (20.7%)
      • seguro_privado: 1,070 personas (0.9%)
      • entidad_prestadora: 753 personas (0.6%)
 

In [23]:
df_final.sample(15)

,conglome,vivienda,hogar,n_personas,n_ninos,n_adultos_mayores,n_edad_trabajar,n_ocupados,n_informales,n_discapacitados,...,tasa_informalidad_hogar,prop_asegurados,prop_discapacitados,ingreso_laboral_percapita,ingreso_por_hora,carga_ninos,carga_adultos_mayores,ocupados_per_capita,es_pobre_monetario,pobreza_laboral
14368,017278,388,11,4,0,0,2,2,2.0,0,...,1.000000,0.250000,0.00,0.000000,0.000000,0.000000,0.00,1.000000,0,1
30826,020126,062,22,3,1,0,2,1,1.0,0,...,1.000000,1.000000,0.00,5341.666504,95.386905,0.333333,0.00,0.333333,1,0
11788,016849,028,11,3,1,0,2,1,0.0,0,...,0.000000,0.666667,0.00,3965.000000,61.953125,0.333333,0.00,0.333333,0,0
14783,017335,111,11,4,0,1,3,2,0.0,1,...,0.000000,1.000000,0.25,11943.750000,140.514706,0.000000,0.25,0.500000,0,0
8518,016358,074,11,4,0,0,4,3,2.0,0,...,0.666667,1.000000,0.00,16994.500000,124.959559,0.000000,0.00,0.750000,0,0
20316,018324,044,11,4,1,0,3,1,0.0,0,...,0.000000,1.000000,0.00,3911.750000,81.494792,0.250000,0.00,0.250000,0,0
438,015078,047,11,7,3,0,4,4,4.0,0,...,1.000000,1.000000,0.00,5753.428711,71.407801,0.428571,0.00,0.571429,0,0
19517,018152,131,11,2,0,0,2,2,1.0,0,...,0.500000,0.500000,0.00,11306.500000,47.110417,0.000000,0.00,1.000000,0,0
21133,018477,020,11,8,0,0,6,3,1.0,0,...,0.333333,0.500000,0.00,5657.500000,72.532051,0.000000,0.00,0.500000,0,0
238,015042,140,11,5,2,0,3,1,1.0,0,...,1.000000,0.800000,0.00,7467.200195,233.350000,0.400000,0.00,0.200000,0,0


In [24]:
df_final.to_csv('dataframe_final_2024.csv', index=False)